# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from xgboost import XGBClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../webinar_3/raw_data/retail_train.csv')
item_features = pd.read_csv('../webinar_3/raw_data/product.csv')
user_features = pd.read_csv('../webinar_3/raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [4]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [5]:
recommender.get_als_recommendations(2500, N=5)

[9420181, 916122, 1085031, 938596, 1139782]

In [6]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [7]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 999270, 1003616, 15778319]

In [8]:
recommender.get_similar_users_recommendation(2500, N=5)

[12487356, 1056212, 1080888, 1057168, 972191]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [5]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
def do_recommend(model, func, n, k):
    
    func_dic = {
        'get_als_recommendations': model.get_als_recommendations,
        'get_own_recommendations': model.get_own_recommendations,
        'get_similar_items_recommendation': model.get_similar_items_recommendation,
        'get_similar_users_recommendation': model.get_similar_users_recommendation
    }
    
    try:
        result = func_dic[func](n, k)
    
    except (IndexError, ValueError) as e:
        
        result = recommender._extend_with_top_popular([], k)
    
    return result

In [11]:
%%time 

k = 50
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_als_recommendations', x, k))

CPU times: user 38.1 s, sys: 1.33 s, total: 39.4 s
Wall time: 21.5 s


In [12]:
%%time

result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_own_recommendations', x, k))

CPU times: user 12.5 s, sys: 920 ms, total: 13.4 s
Wall time: 15.4 s


In [13]:
%%time

result_lvl_1['sim_item'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_similar_items_recommendation', x, k))

CPU times: user 27.1 s, sys: 664 ms, total: 27.8 s
Wall time: 16.2 s


In [14]:
%time

result_lvl_1['sim_user'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_similar_users_recommendation', x, k))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


In [15]:
for col in result_lvl_1.columns[2:]:
    print(f"{col}: {result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], k=k), axis=1).mean()}")


als: 0.04767181333424707
own: 0.06519221251051317
sim_item: 0.03380434905457355
sim_user: 0.006362377358959807


In [16]:
# Ответ:
# наибольший recall  дает вариант get_own_recommendations

In [17]:
k = [20, 50, 100, 200, 500]

tmp_result = pd.DataFrame()

for i in k:
    tmp_result[i] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_own_recommendations', x, i))
    

In [18]:
tmp_result['actual'] = result_lvl_1['actual']

In [19]:
cols = tmp_result.columns[:-1].to_list()
cols

[20, 50, 100, 200, 500]

In [20]:
for col in cols:
    print(f"{col}: {tmp_result.apply(lambda row: recall_at_k(row[col], row['actual'], k=col), axis=1).mean()}")


20: 0.03924282767496872
50: 0.06519221251051317
100: 0.09593769096482338
200: 0.13521077136359616
500: 0.18182621822011844


### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [6]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2.head(2)

,user_id
0,2070
1,2021


In [7]:
# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."
2,1753,"[967041, 963686, 948640, 1057168, 942475, 9421..."
3,2120,"[5707857, 1029743, 1106523, 5569230, 916122, 8..."
4,1346,"[1135983, 5569309, 1129982, 5574377, 5569993, ..."
...,...,...
2149,1446,"[6391134, 12487356, 827332, 7024847, 13007435,..."
2150,1784,"[917406, 1131625, 849578, 950935, 886395, 9796..."
2151,436,"[953180, 918046, 915459, 934399, 1092363, 8787..."
2152,1697,"[993838, 922847, 886103, 7442505, 13133916, 10..."


In [8]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,1097350,1
0,2070,879194,1
0,2070,948640,1


In [9]:
users_lvl_2.shape[0], users_lvl_2['user_id'].nunique()

(107550, 2151)

In [141]:
targets_lvl_2 = data_train_lvl_2.copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [142]:
targets_lvl_2['target'].mean()

0.10460593102333061

#### GB

In [143]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [144]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [145]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [146]:
# добавим средний чек по каждому пользователю

basket_sum = targets_lvl_2.groupby(['user_id', 'basket_id'], as_index=False).agg({'sales_value': 'sum'})
basket_mean = basket_sum.groupby(['user_id'], as_index=False).agg({'sales_value': 'mean'}).\
            rename(columns={'sales_value':'basket_mean'})

targets_lvl_2 = targets_lvl_2.merge(basket_mean, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28


In [147]:
# кол-во покупок в каждой категрии по пользователям

user_cat_sale_count = targets_lvl_2.groupby(['user_id', 'commodity_desc'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_count'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(user_cat_sale_count, on=['user_id', 'commodity_desc'], how='left')
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,3
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,1


In [148]:
# кол-во покупок категории в неделю

commodity_week_count = targets_lvl_2.groupby(['commodity_desc', 'week_no'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_week_count'}).\
                        groupby(['commodity_desc']).agg({'commodity_week_count': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_week_count, on='commodity_desc', how='left')
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,3,2.666667
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,1,6.500000


In [149]:
# стоимость товара к средней стоимости по категории

commodity_mean_price = targets_lvl_2.groupby(['commodity_desc', 'item_id'], as_index=True).\
                        agg({'sales_value': 'mean'}).\
                        rename(columns={'sales_value':'item_price'}).\
                        groupby(['commodity_desc']).agg({'item_price': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_mean_price, on='commodity_desc', how='left')
targets_lvl_2['price_to_cat'] = targets_lvl_2[['sales_value', 'item_price']].\
                    apply(lambda row: row['sales_value']/row['item_price'], axis=1)
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count,item_price,price_to_cat
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-74K,Unknown,Unknown,1,None/Unknown,5.28,3,2.666667,6.196250,NaN
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-74K,Unknown,Unknown,1,None/Unknown,5.28,1,6.500000,13.146875,NaN


In [168]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [166]:
item_features.columns, user_features.columns

(Index(['item_id', 'manufacturer', 'department', 'brand', 'commodity_desc',
        'sub_commodity_desc', 'curr_size_of_product'],
       dtype='object'),
 Index(['age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc', 'user_id'],
       dtype='object'))

In [169]:
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc',
       'sub_commodity_desc', 'curr_size_of_product', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc']

X_train[cat_feats] = X_train[cat_feats].astype('category')

In [170]:
X_train = pd.get_dummies(X_train, columns=cat_feats, drop_first=True)

In [171]:
xgb_model = XGBClassifier(n_estimators=100, max_depth=3,
                          objective='binary:logistic', random_state=42)

In [172]:
%%time 

xgb_model.fit(X_train, y_train)

CPU times: user 14min 24s, sys: 26.3 s, total: 14min 51s
Wall time: 4min 32s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [173]:
train_preds = xgb_model.predict_proba(X_train)

In [174]:
train_preds[:,1]

array([8.7335093e-06, 7.3222473e-06, 7.3222473e-06, ..., 1.3012314e-05,
       1.3012314e-05, 1.3012314e-05], dtype=float32)

In [175]:
targets_lvl_2['own_proba'] = train_preds[:,1]
targets_lvl_2.head(3)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count,item_price,price_to_cat,own_proba
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,1,None/Unknown,5.28,3,2.666667,6.196250,NaN,0.000009
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,1,None/Unknown,5.28,1,6.500000,13.146875,NaN,0.000007
2,2070,879194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,1,None/Unknown,5.28,2,11.166667,11.174832,NaN,0.000007


In [176]:
train_users_lvl2 = targets_lvl_2['user_id'].unique()
train_users_lvl2

array([2070, 2021, 1753, ...,  436, 1697, 1745])

In [177]:
def get_xgb_rec(uid, df):
    return df.loc[df['user_id']==uid][['item_id', 'own_proba']].\
                sort_values('own_proba', ascending=False).head(5).item_id.to_list()

In [178]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']
result_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [179]:
train_users_2 = users_lvl_2['user_id'].unique()
result_lvl_2 = result_lvl_2[result_lvl_2['user_id'].isin(train_users_2)]


In [180]:
result_lvl_2['candidates'] = result_lvl_2['user_id'].apply(lambda x: 
                                                          recommender.get_own_recommendations(x, N=5))
result_lvl_2.head(3)

,user_id,actual,candidates
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 9297615, 5577022, 877391, 9655212]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13003092, 972416, 995598, 923600, 1138596]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[998519, 894360, 7147142, 9338009, 896666]"


In [181]:
result_lvl_2['xgb_candidates'] = result_lvl_2['user_id'].apply(lambda x: 
                                                          get_xgb_rec(x, targets_lvl_2))
result_lvl_2.head(3)

,user_id,actual,candidates,xgb_candidates
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 9297615, 5577022, 877391, 9655212]","[856942, 8293439, 979792, 856942, 9527558]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13003092, 972416, 995598, 923600, 1138596]","[13115971, 12757544, 1098844, 1098844, 13115375]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[998519, 894360, 7147142, 9338009, 896666]","[1122358, 9338009, 870882, 946244, 930918]"


In [182]:
prec_1lvl_model = result_lvl_2['user_id'].apply(lambda x: 
                                    precision_at_k(result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'candidates'].squeeze(), 
                                                   result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

prec_2lvl_model = result_lvl_2['user_id'].apply(lambda x: 
                                    precision_at_k(result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'xgb_candidates'].squeeze(), 
                                                   result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

print(f'1-lvl model Precision is {prec_1lvl_model}\n2-lvl model Precision is {prec_2lvl_model}')

1-lvl model Precision is 0.15070496083550916
2-lvl model Precision is 0.16678851174934725
